In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP),
    RobotConfig(1, RobotType.panda, None,
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


# init stereo aruco detector

In [3]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.83040017, -0.03067266, -0.5563226 ,  0.647911  ],
        [ 0.17474653,  0.9624417 ,  0.20777315, -0.14355281],
        [ 0.52905506, -0.2697503 ,  0.8045716 ,  0.07200202],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## init scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo, base_link="base_link")
s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## add environment

In [6]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

Please create a subscriber to the marker


In [7]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]})

In [8]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add movable

In [9]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

## init planning scene

In [10]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [11]:
from pkg.planning.constraint.constraint_action import Gripper2Tool, PlacePlane

In [12]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [13]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.02), rpy=(0,0,0))

## add objects

In [14]:
from pkg.planning.constraint.constraint_object import BoxAction

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxAction, hexahedral=True)
box2= pscene.create_object(oname="box2", gname="box2", _type=BoxAction, hexahedral=True)

## Register object binders

In [16]:
box1.register_binders(pscene, PlacePlane)
box2.register_binders(pscene, PlacePlane)

## Motion Planning

In [17]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner

In [18]:
mplan = MoveitPlanner(pscene)

In [19]:
from_state = pscene.get_state(crob.home_pose)
print(from_state)

((('box1', 'back_p', 'floor'), ('box2', 'back_p', 'floor')), {'box1': '[[ 0.99 -0.   -0.15 -0.21]\n [ 0.15  0.01  0.99 -0.21]\n [-0.   -1.    0.01  0.03]\n [ 0.    0.    0.    1.  ]]', 'box2': '[[ 0.99 -0.   -0.15 -0.21]\n [ 0.15  0.01  0.99 -0.21]\n [-0.   -1.    0.01  0.03]\n [ 0.    0.    0.    1.  ]]'}, '[ 0.    0.   -1.57  0.   -1.57  0.    0.   -0.39  0.   -1.57  0.    1.57\n  1.57]')


In [20]:
to_state = pscene.make_goal_state(from_state, obj="box1", handle="left_g", binder="grip0")
print(to_state)

((('box1', 'left_g', 'grip0'), ('box2', 'back_p', 'floor')), {'box1': '[[ 0.99 -0.   -0.15 -0.21]\n [ 0.15  0.01  0.99 -0.21]\n [-0.   -1.    0.01  0.03]\n [ 0.    0.    0.    1.  ]]', 'box2': '[[ 0.99 -0.   -0.15 -0.21]\n [ 0.15  0.01  0.99 -0.21]\n [-0.   -1.    0.01  0.03]\n [ 0.    0.    0.    1.  ]]'}, '[ 0.    0.   -1.57  0.   -1.57  0.    0.   -0.39  0.   -1.57  0.    1.57\n  1.57]')


In [22]:
from pkg.planning.constraint.constraint_common import sample_redundancy, combine_redundancy

In [32]:
redundancy = sample_redundancy(
    combine_redundancy(
        pscene.object_dict['box1'].action_points_dict['left_g'], 
        pscene.binder_dict['grip0']))
traj, lastQ, error, success =mplan.plan_transition(from_state, to_state, redundancy_dict={"box1": redundancy})

(array([[ 0.        ,  0.        , -1.57079633,  0.        , -1.57079633,
          0.        ,  0.        , -0.39269908,  0.        , -1.57079633,
          0.        ,  1.57079633,  1.57079633],
        [ 0.08153592,  0.06021258, -1.61100191, -0.03817276, -1.46247406,
          0.04794552,  0.        , -0.39269908,  0.        , -1.57079633,
          0.        ,  1.57079633,  1.57079633],
        [ 0.16307185,  0.12042516, -1.65120749, -0.07634552, -1.35415178,
          0.09589104,  0.        , -0.39269908,  0.        , -1.57079633,
          0.        ,  1.57079633,  1.57079633],
        [ 0.24460777,  0.18063774, -1.69141308, -0.11451829, -1.24582951,
          0.14383656,  0.        , -0.39269908,  0.        , -1.57079633,
          0.        ,  1.57079633,  1.57079633],
        [ 0.32614369,  0.24085032, -1.73161866, -0.15269105, -1.13750724,
          0.19178208,  0.        , -0.39269908,  0.        , -1.57079633,
          0.        ,  1.57079633,  1.57079633],
        [ 0.407

In [ ]:
pscene.gscene.show_motion()

In [17]:
##
# @class    PlanningPipeline
# @brief    planning pipeline
class PlanningPipeline:
    ##
    # @param mplan subclass instance of rnb-planning.src.pkg.planning.motion.interface.MotionInterface
    def set_motion(self, mplan):
        self.mplan = mplan
        mplan.update(self)

    ##
    # @param tplan subclass instance of rnb-planning.src.pkg.planning.task.interface.TaskInterface
    def set_sampler(self, tplan):
        self.tplan = tplan

In [18]:
ghnd.show_pose([0]*13)

## Disconnect stereo

In [20]:
stereo.disconnnect()